### <ins>Código para crear un contador de tableros que dan lugar a una configuración dada.<ins>

Voy a crear un contador de tableros iniciales que dan lugar a un tablero final dado tras un número de generaciones delta. El código está escrito por defecto para tableros 3x3 que lleven, en un paso, a la configuración final:
<pre>
                                                                                    0 0 0
                                                                                    0 1 0
                                                                                    0 0 0
</pre>

Sin embargo, el código está generalizado para cualquier configuración final y cualquier tamaño de tablero.

Este código se ejecuta en la CPU.

Aquí la biblioteca que será de gran ayuda es: 

In [8]:
import itertools
import numpy as np

La librería itertools es un módulo de la biblioteca estándar de Python que proporciona funciones para crear y manipular iteradores de manera eficiente. Estas funciones están pensadas para resolver patrones comunes de iteración, combinatoria y permutaciones de datos. Entre sus funcionalidades destacadas se encuentran:

- **Permutaciones (itertools.permutations):** Genera todas las permutaciones posibles de un iterable (por ejemplo, todas las formas de ordenar una lista).

- **Combinaciones(itertools.combinations / combinations_with_replacement):** Genera combinaciones de elementos de un iterable, ya sea sin reemplazo o con reemplazo.

- **Producto cartesiano (itertools.product):** Crea el producto cartesiano de uno o varios iterables, lo cual es útil para recorrer todas las combinaciones posibles de valores. (Esta es la que nos interesa)

- **Herramientas de conteo y agrupación (chain, groupby, etc.):** Permiten encadenar iteradores o agruparlos bajo ciertas condiciones.

Un aspecto importante de itertools es que la mayoría de sus funciones devuelven iteradores, que consumen menos memoria que las listas y funcionan bajo demanda (lazy evaluation). Esto hace que sea muy útil para recorrer grandes cantidades de datos o combinaciones sin necesitar almacenar todo en memoria simultáneamente.


### 1. Función para generar tableros:

In [9]:
def generar_tableros(size=(3,3)):
    """
    Genera todos los tableros posibles de tamaño size.

    Inputs:

        size = (int,int):  dimensiones de los tableros atendiendo al número de (filas, columnas). Por defecto: (3,3)

    Outputs:
    
        tableros (iterador): configuraciones de todos los tableros iniciales. /!\ Solo se puede llamar una vez, a no ser que se convierta en lista, aunque es más eficiente en memoria como iterador.

    
    Ejemplo:

    size = (3,3) ---> 512 tableros (2^(3*3) = 512)

    """

    r, c = size

    tableros = itertools.product([0,1], repeat= r*c)
    
    return tableros

### 2. Función para contar células vivas entorno a una célula:

In [10]:
def contador_vecinas_vivas(tablero_inicial, r, c, size=(3,3)):
    
    """ 
    Cuenta el número de células vecinas vivas para la célula (r,c) de un tablero dado.

    Inputs:

        tablero_inicial (2d-nparray): configuración del tablero inicial.
        r (int): fila de la célula a analizar.
        c (int): columna de la célula a analizar.
        size (int,int):  dimensiones de los tableros atendiendo al número de (filas, columnas). Por defecto: (3,3)

    Output:
        vivas (int): número de células vecinas vivas.
        
    """
    max_r, max_c = size
    vivas = 0
    for dr in [-1,0,1]:     # dr: desplazamientos en las filas (-1: mov arriba; 0: no mov; 1: mov abajo)
        for dc in [-1,0,1]: # dc: desplazmientos en columnas (-1: mov izq; 0: no mov; 1: mov dcha)
            
            if dr == 0 and dc == 0: # Estariamos sobre la célula (r,c)
                continue
            nr, nc = r + dr, c + dc

            if 0 <= nr < max_r and 0 <= nc < max_c: # Nos aseguramos que nos encontremos en el tablero
                vivas += tablero_inicial[nr][nc]# Como tablero está formado por 0 y 1, solo suma cuando 1 (viva)
    
    return vivas
        

### 3. Función del juego de Conway:

In [11]:
def game_of_life(tablero_inicial, size=(3,3), delta=1):

    """ 
    Función para ejecutar el Juego de Conway sobre un tablero inicial, de tamaño size, delta generaciones.

    Inputs:

        tablero_inicial (2d-nparray): matriz de la configuración del tablero inicial.
        size (int,int):  dimensiones de los tableros atendiendo al número de (filas, columnas). Por defecto: (3,3)
        delta (int): número de pasos entre el tablero inicial y final. (Nº de generaciones)
    
    Outputs:

        tablero_GoL (2d-nparray): matriz de la configuración del tablero final. 
        
    """

    max_r, max_c = size

    next_generation = np.zeros(size)
    tablero = tablero_inicial

    for pasos in range(delta): # Repetimos tantas generaciones como se han indicado:

        for r in range(max_r):  # Barremos todas las filas

            for c in range(max_c): # Barremos todas las columnas

                # Contamos las celdas vivas en torno a la célula (r,c)
                vivas = contador_vecinas_vivas(tablero, r, c, size)

                if tablero[r][c] == 1: # la célula (r,c) está viva:

                    if vivas in [2,3]: # Sobrevive
                        next_generation[r][c] = 1

                else: # la célula (r,c) está muerta:

                    if vivas == 3: # Nace
                        next_generation[r][c] = 1
        tablero = next_generation

    tablero_GoL = next_generation

    return  tablero_GoL 

### 5. Contador Tableros que son idénticos al tablero final deseado:

In [12]:
def contador_tableros(tablero_final=np.array([[0,0,0],[0,1,0],[0,0,0]], dtype=int), size=(3,3), delta = 1):
    """
    Esta función cuenta el número de tableros iniciales, de tamaño size, que llevan a un mismo tablero final, de tamaño size, tras un número de generaciones delta.

    Inputs:
    
        tablero_final (2d-nparray): configuración del tablero final. Por defecto es el tablero 3x3 (0,0,0,0,1,0,0,0,0)
        size (int,int):  dimensiones de los tableros atendiendo al número de (filas, columnas). Por defecto: (3,3)
        delta (int): número de pasos entre el tablero inicial y final. (Nº de generaciones)

    Outputs:
        tablero_final (2d-nparray): configuración del tablero final. Por defecto es el tablero 3x3 (0,0,0,0,1,0,0,0,0)
        antecesores (2d-nparray): matriz donde cada fila representa un tablero inicial que lleva al tablero final marcado.
        contador (int):  número de tableros iniciales que llevan al tablero final dado.
            
    """ 
    tablero_final = np.array(tablero_final).reshape(size) # Nos aseguramos de que se de como 2d-nparray

    tableros = generar_tableros(size) # Generamos los tableros.

    # Inicializamos parámetros:
    contador = 0
    antecesores = []

    for tablero_inicial in tableros:
        tablero_inicial = np.array(tablero_inicial, dtype=int).reshape(size)

        tablero_GoL = np.array(game_of_life(tablero_inicial, size, delta), dtype=int).reshape(size) # Juego de Conway sobre el tablero_inicial generado.
        
        if np.array_equal(tablero_final, tablero_GoL): # Este tablero inicial generado lleva al tablero final dado.
            contador += 1
            antecesores.append(tablero_inicial)
        
    return tablero_final, antecesores, contador

### Prueba de la función:

In [13]:
tablero_final = np.array((0,0,0,0,1,0,0,0,0)).reshape(3,3)
size = (3,3)
delta = 1

tablero_final, antecesores, contador = contador_tableros(tablero_final, size, delta)

print('Hay ', int(contador), ' tableros iniciales que llevan a la configuración final en',int(delta), 'pasos:\n')
print(tablero_final)



Hay  22  tableros iniciales que llevan a la configuración final en 1 pasos:

[[0 0 0]
 [0 1 0]
 [0 0 0]]
